In [4]:
import pandas as pd
import numpy as np 
import os

In [6]:
credits_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/credits.csv'))
keywords_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/keywords.csv'))
links_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/links.csv'))
movies_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/movies_metadata.csv'))
ratings_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/ratings.csv'))

C:\Users\leish\AppData\Local\Temp\ipykernel_29272\3811994195.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/movies_metadata.csv'))


In [7]:
movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')
movies_df = movies_df[movies_df['id'].notna()].astype({'id': int})

In [8]:
links_df.rename(columns={'movieId':'id'},inplace=True)

In [9]:
grouped_ratings_df = ratings_df.groupby('movieId').mean()
grouped_ratings_df.reset_index(inplace=True)
grouped_ratings_df.rename(columns={"movieId":"id"}, inplace=True)

In [10]:
df = movies_df.merge(keywords_df, on='id')
df = df.merge(credits_df[['cast','crew','id']], on='id')
df = df.merge(grouped_ratings_df[['id','rating']], on='id')
df = df.merge(links_df, on='id')

In [11]:
# Creates seperate dataframe that contains all information for a specific column of that dataframe
# The column converts the JSON or array of JSONs of each row into a row to be inserted into the DataFrame
def create_info_dataframe(df,column_header):
    if "[" in df[df[column_header].notna()][column_header].iloc[0]:  
        return pd.DataFrame([x for sublist in df[column_header].map(lambda x: eval(x)).to_numpy().flatten() for x in sublist]).drop_duplicates()
    else:
        return pd.DataFrame([x for x in df[column_header].map(lambda x: eval(x) if isinstance(x, str) and isinstance(eval(x), dict) else eval(
            "{'id': 0,'name':'None','poster_path':'None', 'backdrop_path':'None'}"))])

In [12]:
# Extracts the id of each JSON in each cell and creates an array of these ids to be inserted into the original DataFrame df. 
# The Id of the JSON can reference the additional DataFramescd 
def reduce_json_to_id(df, column_header):
    if "[" in df[df[column_header].notna()][column_header].iloc[0]: 
        return df[column_header].map(lambda x: [y['id'] for y in eval(x)] if isinstance(x, str) else None)
    else:
        return df[column_header].map(lambda x: eval(x)['id'] if isinstance(x, str) and isinstance(eval(x), dict) else None)

In [13]:
genres_df = create_info_dataframe(df, 'genres')
keywords2_df = create_info_dataframe(df, 'keywords')
cast_df = create_info_dataframe(df, 'cast')
crew_df = create_info_dataframe(df, 'crew')
collection_df = create_info_dataframe(df, 'belongs_to_collection')
production_company_df = create_info_dataframe(df, 'production_companies')

In [14]:
df['genres'] = reduce_json_to_id(df, 'genres')
df['keywords'] = reduce_json_to_id(df, 'keywords')
df['cast'] = reduce_json_to_id(df, 'cast')
df['crew'] = reduce_json_to_id(df, 'crew')
df['belongs_to_collection'] = reduce_json_to_id(df, 'belongs_to_collection')
df['production_companies'] = reduce_json_to_id(df, 'production_companies')

In [15]:
df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/master.csv'))
genres_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/genres.csv'))
cast_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/cast.csv'))
crew_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/crew.csv'))
collection_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/collection.csv'))
production_company_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/production_companies.csv'))